In [1]:
import pandas as pd

In [2]:
import io
data = open('./tRNA-modifications.txt').read().split('#')
mod_df = pd.DataFrame()
for d in data:
    mod_df = pd.concat([mod_df, pd.read_csv(io.StringIO(d), delim_whitespace=True).T],join='outer')
all_paper_trnas = set([i.split('.')[0] for i in mod_df.index])

# Targets

In [4]:
from Bio import SeqIO
trna_df = pd.read_csv('tRNAs.txt',sep='\t',index_col=0)
paper_dct = trna_df.reset_index().set_index('Codon')['tRNA'].to_dict()
gb_file = Bio.SeqIO.parse("../building_data/genome_modified.gb", "gb")
contigs = [ i for i in gb_file ]
dct = {'c':'G','g':'C','t':'A','a':'T'}

In [5]:
gb_dct = {}
for c in contigs:
    for f in c.features:
        if f.type == 'tRNA':
            s = f.qualifiers['anticodon'][0].split('seq:')[1].split(')')[0][::-1]
            s2 = ''.join([dct[i] for i in s])
            i = f.qualifiers['old_locus_tag'][0]
            aa = f.qualifiers['product'][0].split('tRNA-')[1]
#             if 'Ala' in f.qualifiers['product'][0]:
            gb_dct[s2] = i
            print(i,aa,s2)

TM_t01 Pro CCA
TM_t02 Ile ATC
TM_t03 Ala GCA
TM_t04 Ala GCG
TM_t05 Arg CGA
TM_t06 His CAC
TM_t07 Arg AGA
TM_t08 Gly GGA
TM_t09 Ser AGC
TM_t10 Arg CGC
TM_t11 Thr ACC
TM_t12 Gln CAG
TM_t13 Met ATG
TM_t14 Met ATG
TM_t15 Thr ACA
TM_t16 Tyr TAC
TM_t17 Trp TGG
TM_t18 Ser TCA
TM_t19 Gly GGC
TM_t20 Leu CTA
TM_t21 Lys AAA
TM_t22 Thr ACG
TM_t23 Gln CAA
TM_t24 Leu CTG
TM_t25 Ser TCG
TM_t26 Leu TTG
TM_t27 Asn AAC
TM_t28 Pro CCG
TM_t29 Met ATG
TM_t30 Gly GGG
TM_t31 Leu TTA
TM_t32 Cys TGC
TM_t33 Val GTC
TM_t34 Leu CTC
TM_t35 Val GTG
TM_t36 Arg CGG
TM_t37 Val GTA
TM_t38 Phe TTC
TM_t39 Ala GCC
TM_t40 Arg AGG
TM_t41 Lys AAG
TM_t42 Glu GAG
TM_t43 Pro CCC
TM_t44 Asp GAC
TM_t45 Ser TCC
TM_t46 Glu GAA


In [6]:
paper_to_gb = {}
for c in gb_dct:
    gb_trna = gb_dct[c]
    paper_trna = paper_dct[c]
    paper_to_gb[paper_trna] = gb_trna

In [7]:
coralme_rnamod = pd.DataFrame()

In [8]:
for i in all_paper_trnas:
    if i not in paper_to_gb:
        print(i)
        continue
    trna = paper_to_gb[i]
    mods = mod_df[mod_df.index.str.contains(i)]
    rnamod_dct = {}
    for j,row in mods.iterrows():
        rnamod_dct[trna] = {'position':row[1],
                        'modification':row[0]}
        coralme_rnamod = pd.concat([coralme_rnamod, pd.DataFrame.from_dict(rnamod_dct).T],join='outer')

TMtRNA-Met-1
TMtRNA-Met-2


In [ ]:
coralme_rnamod

In [119]:
# coralme_rnamod.to_csv('./building_data/post_transcriptional_modification_of_RNA.txt',sep='\t')

# Machinery

In [151]:
dct = {}
for i,row in mod_df.iterrows():
    i = i.split('.')[0]
    if i not in paper_to_gb:
        print(i)
        continue
    if 'Monomer' not in row[2]:
        continue
    trna = paper_to_gb[i]
    enz = row[2].split('_assumed')[0]
    if enz not in dct:
        dct[enz] = set()
    dct[enz].add('{}_at_{}'.format(row[1],row[0]))


TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-1
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2
TMtRNA-Met-2


In [154]:
for k,v in dct.items():
    mods = [i.split('_at_')[1] for i in v]
    pos = [i.split('_at_')[0] for i in v]
    print(k,mods,','.join(pos))

TM0748 ['m1A'] 58
TM0856 ['Y'] 55
TM0925 ['m7G'] 46
TM0096 ['D', 'D', 'D', 'D', 'D', 'D', 'D'] 20,22,18,17,19,21,16
TM1574 ['Y', 'Y'] 39,40
TM0852 ['t6A'] 37


## ATPM

Supplementary information of the paper states atp maintenance is 25.0

https://static-content.springer.com/esm/art%3A10.1038%2Fncomms1928/MediaObjects/41467_2012_BFncomms1928_MOESM921_ESM.pdf

## New reactions

In [1]:
import cobra

/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


In [3]:
model = cobra.io.read_sbml_model('./tmaritima-ME.xml')

Exception: libSBML required for fbc < 2

In [ ]:
for r in model.metabolites.atp_c.reactions:
    if r.lower_bound > 0:
        print(r.id) 
        print(r.reaction)